In [ ]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = None 
pd.options.display.max_rows = None

import timeit

#### workflow: convert to games data, get year column, get dummies, random split in train test with sklearn train test split 

In [2]:
data = pd.read_csv('../Data/NFL_Play_by_Play_2009-2018(v5).csv')

/Users/ulrikeanklam/opt/anaconda3/envs/matrix/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
def clean_data(df):
    df = df.assign(field_goal = np.where(df['field_goal_result'] == 'made', 1, 0))
    df = df.assign(extra_point = np.where(df['extra_point_result'] == 'good', 1, 0))
    df = df.assign(two_point_conversion = np.where(df['two_point_conv_result'] == 'success', 1, 0))
    
    missing_values = df.isnull().sum()
    mv_ratio = missing_values[missing_values.gt(0)]/len(df)
    list_mv_50 = mv_ratio.loc[mv_ratio.gt(0.5)].index
    
    train_01 = df.drop( columns = list_mv_50).copy()
    
    mv_01 = train_01.isnull().sum()
    list_cols = mv_01.loc[mv_01.gt(0)].index
    
    for col in list_cols:
        train_01[col] = train_01[col].fillna(0)
    
    return train_01

In [6]:
def to_gamesdata(df):
    df['succesful_play'] = (np.where((df['play_type'] == 'pass')
                                      & (df['yards_gained']>= 6.3), 1,
                                 np.where((df['play_type'] == 'run') 
                                          & (df['yards_gained'] >= 4.4), 1, 0)))
    
    df['posteam_is_home'] = (np.where(df['posteam_type'] == 'home', 1 , 0))
    
    playtype = pd.get_dummies(df['play_type'], prefix= 'playtype', drop_first = True )
    df = df.join(playtype)
    
    df['side_of_field_is_hometeam'] = np.where(df['side_of_field'] == df['home_team'], 1, 0)
    
    object_cols = ['posteam', 'posteam_type','defteam',
                   'side_of_field','game_date','time','yrdln', 'desc','play_type']
    df = df.drop(columns = object_cols)
    
    games = df.groupby(['game_id','posteam_is_home', 'home_team', 'away_team'], as_index = False ).sum()
    
    games = games[['game_id', 'posteam_is_home', 'home_team', 'away_team','succesful_play',
               'shotgun', 'no_huddle', 'punt_blocked',
                'first_down_rush', 'first_down_pass',
               'first_down_penalty', 'third_down_converted', 'third_down_failed', 'fourth_down_converted',
               'fourth_down_failed', 'interception', 'safety', 'penalty', 'tackled_for_loss', 
              'fumble_lost', 'incomplete_pass', 'qb_hit', 'sack', 'rush_touchdown' , 'pass_touchdown',
               'return_touchdown','field_goal' ,'extra_point', 'two_point_conversion']]
    
    games_home = games.loc[games['posteam_is_home'] == 1]
    games_away = games.loc[games['posteam_is_home'] == 0]
    
    gamedata = games_home.merge(games_away,how = 'inner', on = games_home['game_id'], 
                            suffixes = ('_hometeam', '_awayteam') )
    
    gamedata = gamedata.drop(columns = ['key_0', 'posteam_is_home_hometeam', 'game_id_awayteam',
                                    'posteam_is_home_awayteam','home_team_awayteam', 'away_team_awayteam' ])
    
    dummies_home = pd.get_dummies(gamedata['home_team_hometeam'], prefix= 'hometeam', drop_first = True)
    dummies_away = pd.get_dummies(gamedata['away_team_hometeam'], prefix= 'awayteam', drop_first = True)
    
    gamedata = gamedata.join([dummies_home, dummies_away])
    
    gamedata = gamedata.drop(columns = ['home_team_hometeam' , 'away_team_hometeam'])
    
    gamedata = gamedata.assign( total_points_hometeam = ((gamedata['rush_touchdown_hometeam'] * 6)
                                                     + (gamedata['pass_touchdown_hometeam'] * 6)
                                                     + (gamedata['return_touchdown_hometeam'] * 6)
                                                     + (gamedata['field_goal_hometeam'] * 3) 
                                                     + (gamedata['extra_point_hometeam'])
                                                     + (gamedata['two_point_conversion_hometeam'] * 2)))
    
    gamedata = gamedata.assign( total_points_awayteam = ((gamedata['rush_touchdown_awayteam'] * 6)
                                                     + (gamedata['pass_touchdown_awayteam'] * 6)
                                                     + (gamedata['return_touchdown_awayteam'] * 6)
                                                     + (gamedata['field_goal_awayteam'] * 3) 
                                                     + (gamedata['extra_point_awayteam'])
                                                     + (gamedata['two_point_conversion_awayteam'] * 2)))
    
    gamedata = gamedata.assign(hometeam_is_winner = np.where(gamedata['total_points_hometeam'] > gamedata['total_points_awayteam'],
                                                        1, 0))
    return gamedata

In [7]:
# cleaning original dataset and processing it to games data
games_data = to_gamesdata(clean_data(data))

In [8]:
games_data.shape

(2526, 122)

In [9]:
games_data.dtypes

game_id_hometeam                    int64
succesful_play_hometeam             int64
shotgun_hometeam                    int64
no_huddle_hometeam                  int64
punt_blocked_hometeam             float64
first_down_rush_hometeam          float64
first_down_pass_hometeam          float64
first_down_penalty_hometeam       float64
third_down_converted_hometeam     float64
third_down_failed_hometeam        float64
fourth_down_converted_hometeam    float64
fourth_down_failed_hometeam       float64
interception_hometeam             float64
safety_hometeam                   float64
penalty_hometeam                  float64
tackled_for_loss_hometeam         float64
fumble_lost_hometeam              float64
incomplete_pass_hometeam          float64
qb_hit_hometeam                   float64
sack_hometeam                     float64
rush_touchdown_hometeam           float64
pass_touchdown_hometeam           float64
return_touchdown_hometeam         float64
field_goal_hometeam               

In [13]:
# adding a column for the year/season
games_data = games_data.assign(year = games_data['game_id_hometeam'].apply(str).str[:4].apply(int))

In [14]:
# games_data.head()

,game_id_hometeam,succesful_play_hometeam,shotgun_hometeam,no_huddle_hometeam,punt_blocked_hometeam,first_down_rush_hometeam,first_down_pass_hometeam,first_down_penalty_hometeam,third_down_converted_hometeam,third_down_failed_hometeam,fourth_down_converted_hometeam,fourth_down_failed_hometeam,interception_hometeam,safety_hometeam,penalty_hometeam,tackled_for_loss_hometeam,fumble_lost_hometeam,incomplete_pass_hometeam,qb_hit_hometeam,sack_hometeam,rush_touchdown_hometeam,pass_touchdown_hometeam,return_touchdown_hometeam,field_goal_hometeam,extra_point_hometeam,two_point_conversion_hometeam,succesful_play_awayteam,shotgun_awayteam,no_huddle_awayteam,punt_blocked_awayteam,first_down_rush_awayteam,first_down_pass_awayteam,first_down_penalty_awayteam,third_down_converted_awayteam,third_down_failed_awayteam,fourth_down_converted_awayteam,fourth_down_failed_awayteam,interception_awayteam,safety_awayteam,penalty_awayteam,tackled_for_loss_awayteam,fumble_lost_awayteam,incomplete_pass_awayteam,qb_hit_awayteam,sack_awayteam,rush_touchdown_awayteam,pass_touchdown_awayteam,return_touchdown_awayteam,field_goal_awayteam,extra_point_awayteam,two_point_conversion_awayteam,hometeam_ATL,hometeam_BAL,hometeam_BUF,hometeam_CAR,hometeam_CHI,hometeam_CIN,hometeam_CLE,hometeam_DAL,hometeam_DEN,hometeam_DET,hometeam_GB,hometeam_HOU,hometeam_IND,hometeam_JAC,hometeam_JAX,hometeam_KC,hometeam_LA,hometeam_LAC,hometeam_MIA,hometeam_MIN,hometeam_NE,hometeam_NO,hometeam_NYG,hometeam_NYJ,hometeam_OAK,hometeam_PHI,hometeam_PIT,hometeam_SD,hometeam_SEA,hometeam_SF,hometeam_STL,hometeam_TB,hometeam_TEN,hometeam_WAS,awayteam_ATL,awayteam_BAL,awayteam_BUF,awayteam_CAR,awayteam_CHI,awayteam_CIN,awayteam_CLE,awayteam_DAL,awayteam_DEN,awayteam_DET,awayteam_GB,awayteam_HOU,awayteam_IND,awayteam_JAC,awayteam_JAX,awayteam_KC,awayteam_LA,awayteam_LAC,awayteam_MIA,awayteam_MIN,awayteam_NE,awayteam_NO,awayteam_NYG,awayteam_NYJ,awayteam_OAK,awayteam_PHI,awayteam_PIT,awayteam_SD,awayteam_SEA,awayteam_SF,awayteam_STL,awayteam_TB,awayteam_TEN,awayteam_WAS,total_points_hometeam,total_points_awayteam,hometeam_is_winner,year
0,2009091000,27,38,14,0.0,1.0,18.0,0.0,4.0,10.0,0.0,0.0,2.0,0.0,5.0,3.0,1.0,8.0,7.0,4.0,0.0,1.0,0.0,2,1,0,20,18,0,0.0,2.0,14.0,2.0,4.0,9.0,0.0,0.0,1.0,0.0,6.0,5.0,1.0,12.0,3.0,1.0,0.0,1.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,13.0,10.0,1,2009
1,2009091300,23,17,0,0.0,3.0,15.0,1.0,6.0,9.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,14.0,2.0,2.0,0.0,2.0,0.0,2,1,0,21,21,0,0.0,4.0,11.0,1.0,4.0,7.0,1.0,0.0,1.0,0.0,4.0,2.0,3.0,8.0,4.0,4.0,0.0,1.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.0,7.0,1,2009
2,2009091301,37,20,43,1.0,11.0,20.0,1.0,10.0,7.0,1.0,0.0,1.0,0.0,5.0,1.0,0.0,16.0,0.0,1.0,2.0,3.0,0.0,1,5,0,12,23,2,0.0,1.0,8.0,2.0,2.0,8.0,1.0,1.0,0.0,0.0,6.0,5.0,0.0,8.0,2.0,3.0,0.0,2.0,0.0,1,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.0,17.0,1,2009
3,2009091302,16,24,5,0.0,4.0,8.0,2.0,3.0,13.0,2.0,1.0,5.0,0.0,6.0,4.0,2.0,15.0,9.0,5.0,1.0,0.0,1.0,1,1,0,20,26,2,0.0,9.0,7.0,1.0,5.0,8.0,1.0,1.0,1.0,0.0,8.0,1.0,1.0,11.0,1.0,2.0,1.0,2.0,0.0,1,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,16.0,24.0,0,2009
4,2009091303,25,23,0,0.0,4.0,12.0,0.0,5.0,10.0,1.0,1.0,2.0,0.0,4.0,3.0,0.0,10.0,4.0,3.0,1.0,0.0,0.0,0,1,0,19,31,0,0.0,3.0,7.0,0.0,3.0,9.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,11.0,5.0,3.0,0.0,1.0,0.0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.0,12.0,0,2009


In [15]:
games_data.year.value_counts()

2017    272
2012    271
2013    256
2011    256
2014    256
2010    256
2016    255
2015    240
2009    240
2018    224
Name: year, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split

In [ ]:
X_cols = [col for col in train.columns if not col in ['total_points_hometeam',
       'total_points_awayteam', 'hometeam_is_winner'] ]
X = games_data[X_cols]
y = games_data['hometeam_is_winner']

In [18]:
train , test = train_test_split(games_data, test_size = 0.2)

In [19]:
train.shape

(2020, 123)

In [20]:
test.shape

(506, 123)

In [22]:
# exporting train dataframe
#export_train = train.to_csv('../Data/train_games.csv', index = False)

In [23]:
# exporting test dataframe
# export_test = test.to_csv('../Data/test_games.csv', index = False)